In [1]:
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from bs4 import BeautifulSoup as bs
import time
import requests
import pymongo
from splinter import Browser
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

## Step 1 - Scraping
### NASA Mars News
Scrape the [Mars News Site](https://redplanetscience.com/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\Tang\.wdm\drivers\chromedriver\win32\91.0.4472.19\chromedriver.exe] found in cache


In [3]:
url = "https://redplanetscience.com"
browser.visit(url)
time.sleep(1)
html = browser.html
soup = bs(html, "html.parser")


In [4]:
# Retrieve the latest news title
news_title=soup.find_all('div', class_='content_title')[0].text
# Retrieve the latest news paragraph
news_p=soup.find_all('div', class_='article_teaser_body')[0].text
print(news_title)
print(16*("-"))
print(news_p)

NASA's Treasure Map for Water Ice on Mars
----------------
A new study identifies frozen water just below the Martian surface, where astronauts could easily dig it up.


### JPL Mars Space Images - Featured Image
* Visit the url for the Featured Space Image site [here](https://spaceimages-mars.com).

* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.

* Make sure to find the image url to the full size `.jpg` image.

* Make sure to save a complete url string for this image.

In [5]:
jpl_url = "https://spaceimages-mars.com/"
browser.visit(jpl_url)
#Find featured image
featured_image_url = browser.find_by_tag('img[class="headerimage fade-in"]')[0]["src"]
print(featured_image_url)



https://spaceimages-mars.com/image/featured/mars3.jpg


## Mars Facts
* Visit the Mars Facts webpage [here](https://galaxyfacts-mars.com) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

* Use Pandas to convert the data to a HTML table string.

In [26]:
facts_url = "https://galaxyfacts-mars.com"
browser.visit(facts_url)
facts_table = pd.read_html(facts_url, header=0, index_col=0)
facts_table[0]



,Mars,Earth
Mars - Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C




### Mars Hemispheres

* Visit the astrogeology site [here](https://marshemispheres.com/) to obtain high resolution images for each of Mar's hemispheres.

* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.

* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [ ]:
hemi_url = "https://marshemispheres.com/"
browser.visit(hemi_url)




## Step 2 - MongoDB and Flask Application

Use MongoDB with Flask templating to create a new HTML page that displays all of the information that was scraped from the URLs above.

* Start by converting your Jupyter notebook into a Python script called `scrape_mars.py` with a function called `scrape` that will execute all of your scraping code from above and return one Python dictionary containing all of the scraped data.

* Next, create a route called `/scrape` that will import your `scrape_mars.py` script and call your `scrape` function.

  * Store the return value in Mongo as a Python dictionary.

* Create a root route `/` that will query your Mongo database and pass the mars data into an HTML template to display the data.

* Create a template HTML file called `index.html` that will take the mars data dictionary and display all of the data in the appropriate HTML elements. Use the following as a guide for what the final product should look like, but feel free to create your own design.